In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.layers import Activation, Dense, Embedding, SimpleRNN, LSTM
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from sklearn.preprocessing import LabelEncoder
import time
from keras import metrics
print('import done')

import done


In [7]:
DATA_FILE = '/content/drive/My Drive/Colab Notebooks/sentiment-analysis-on-movie-reviews/train.tsv/train.tsv'
train = pd.read_csv(DATA_FILE,sep='\t', header = 0)
tags = train.Sentiment
texts = train.Phrase

num_max = 1000
# preprocess
le = LabelEncoder()
tags = le.fit_transform(tags)
tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(texts)
mat_texts = tok.texts_to_matrix(texts,mode='count')
print(tags[:5])
print(mat_texts[:5])
print(tags.shape,mat_texts.shape)

[1 2 2 2 2]
[[0. 3. 2. ... 0. 0. 0.]
 [0. 2. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(156060,) (156060, 1000)


In [8]:
from keras.preprocessing import sequence

def check_model(model,x,y):
    model.fit(x,y,batch_size=32,epochs=10,verbose=1,validation_split=0.2)


max_len = 100
cnn_texts_seq = tok.texts_to_sequences(texts)
print(cnn_texts_seq[0])
cnn_texts_mat = sequence.pad_sequences(cnn_texts_seq,maxlen=max_len)
print(cnn_texts_mat[0])
print(cnn_texts_mat.shape)

[2, 323, 3, 1, 9, 52, 8, 46, 13, 1, 8, 177, 46, 13, 1, 65, 3, 78, 668, 19, 576, 3, 78, 5, 57, 3, 2, 42]
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   2 323   3   1   9  52   8  46  13   1   8 177  46  13   1  65   3  78
 668  19 576   3  78   5  57   3   2  42]
(156060, 100)


In [9]:
def get_rnn_model_v1():
    rnn_model = Sequential()
    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    # 1000 is num_max
    rnn_model.add(Embedding(1000,
                            20,
                            input_length=max_len))

    rnn_model.add(SimpleRNN(32))
    rnn_model.add(Dense(1))
    rnn_model.add(Activation('sigmoid'))
    rnn_model.summary()

    rnn_model.compile(optimizer="adam",
                      loss='binary_crossentropy',
                      metrics=['accuracy'])

    return rnn_model

m = get_rnn_model_v1()
check_model(m,cnn_texts_mat,tags)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 20)           20000     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                1696      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 21,729
Trainable params: 21,729
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 124848 samples, validate on 31212 samples
Epoch 1/10
124848/124848 [==============================] - 83s 668us/step - loss: -75.3915 - accuracy: 0.1710 - val_loss: -141.5939 - val_accuracy: 0.1896
Epoch 2/10
124848/124848 [==============================] - 72s 576us/step - loss: -211.7798 - accuracy: 0.1710 - val_loss: -275.3965 - val_accuracy: 0.1896
Epoch 3/10
124848/124848 [==============================] - 74s 594us/step - loss: -347.6575 - accuracy: 0.1710 - val_loss: -409.2501 - val_accuracy: 0.1896
Epoch 4/10
124848/124848 [==============================] - 68s 544us/step - loss: -484.0282 - accuracy: 0.1712 - val_loss: -543.2821 - val_accuracy: 0.1897
Epoch 5/10
124848/124848 [==============================] - 66s 527us/step - loss: -620.0944 - accuracy: 0.1712 - val_loss: -676.9605 - val_accuracy: 0.1898
Epoch 6/10
124848/124848 [==============================] - 66s 530us/step - loss: -756.2844 - accuracy: 0.1712 - val_loss: -810.8611 - val_accuracy: 0.1899
Epoch 7/